In [14]:
import jsonlines
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
matches = []
with jsonlines.open('matches.jl') as reader:
    for obj in reader:
        matches.append(obj)

In [3]:
len(matches)

4422

In [4]:
matches[0]

{'team1': {'name': 'OG', 'points': 2},
 'lineup1': [{'name': 'valde',
   'rating': 1.09,
   'DPR': 0.65,
   'KAST': 0.725,
   'impact': 1.11,
   'ADR': 80.5,
   'KPR': 0.74},
  {'name': 'Aleksib',
   'rating': 0.97,
   'DPR': 0.66,
   'KAST': 0.713,
   'impact': 1.05,
   'ADR': 75.5,
   'KPR': 0.65},
  {'name': 'niko',
   'rating': 1.03,
   'DPR': 0.66,
   'KAST': 0.7140000000000001,
   'impact': 1.05,
   'ADR': 75.6,
   'KPR': 0.7},
  {'name': 'mantuu',
   'rating': 1.16,
   'DPR': 0.63,
   'KAST': 0.727,
   'impact': 1.18,
   'ADR': 77.7,
   'KPR': 0.75},
  {'name': 'flameZ',
   'rating': 1.14,
   'DPR': 0.67,
   'KAST': 0.715,
   'impact': 1.18,
   'ADR': 80.8,
   'KPR': 0.74}],
 'team2': {'name': 'Fiend', 'points': 0},
 'lineup2': [{'name': 'bubble',
   'rating': 1.01,
   'DPR': 0.65,
   'KAST': 0.716,
   'impact': 1.04,
   'ADR': 75.1,
   'KPR': 0.68},
  {'name': 'v1c7oR',
   'rating': 1.03,
   'DPR': 0.67,
   'KAST': 0.73,
   'impact': 1.12,
   'ADR': 77.2,
   'KPR': 0.7},
  {'na

In [17]:
head = ["team1_name", "team2_name", "best_of", "match_type", "date", "timestamp", "prize", "teams", "location"]
head += [f"map{i}" for i in range(1,6)]
head += [f"team{team}_player{p}_{attr}" for team in range(1,3) for p in range(1,6) for attr in ["name", "rating", "DPR", "KAST", "impact", "ADR", "KPR"]]
head += ["team1_points", "team2_points", "team1_wins", "team2_wins"]
head

['team1_name',
 'team2_name',
 'best_of',
 'match_type',
 'date',
 'timestamp',
 'prize',
 'teams',
 'location',
 'map1',
 'map2',
 'map3',
 'map4',
 'map5',
 'team1_player1_name',
 'team1_player1_rating',
 'team1_player1_DPR',
 'team1_player1_KAST',
 'team1_player1_impact',
 'team1_player1_ADR',
 'team1_player1_KPR',
 'team1_player2_name',
 'team1_player2_rating',
 'team1_player2_DPR',
 'team1_player2_KAST',
 'team1_player2_impact',
 'team1_player2_ADR',
 'team1_player2_KPR',
 'team1_player3_name',
 'team1_player3_rating',
 'team1_player3_DPR',
 'team1_player3_KAST',
 'team1_player3_impact',
 'team1_player3_ADR',
 'team1_player3_KPR',
 'team1_player4_name',
 'team1_player4_rating',
 'team1_player4_DPR',
 'team1_player4_KAST',
 'team1_player4_impact',
 'team1_player4_ADR',
 'team1_player4_KPR',
 'team1_player5_name',
 'team1_player5_rating',
 'team1_player5_DPR',
 'team1_player5_KAST',
 'team1_player5_impact',
 'team1_player5_ADR',
 'team1_player5_KPR',
 'team2_player1_name',
 'team2_p

In [19]:
records = []
for match in matches:
    record = []
    record.append(match["team1"]["name"])
    record.append(match["team2"]["name"])
    record.append(match["best_of"])
    record.append(match["match_type"])
    record.append(match["date"])
    record.append(match["unix_time"])
    record.append(match["event"]["prize"])
    record.append(match["event"]["teams"])
    record.append(match["event"]["location"])
    maps = match["maps"]
    for i in range(5):
        if i < len(maps):
            record.append(maps[i])
        else:
            record.append(None)
    for player in match["lineup1"]:
        for attr in player.keys():
            record.append(player[attr])
    for player in match["lineup2"]:
        for attr in player.keys():
            record.append(player[attr])
    record.append(match["team1"]["points"])
    record.append(match["team2"]["points"])
    record.append(match["team1"]["points"] > match["team2"]["points"])
    record.append(match["team2"]["points"] > match["team1"]["points"])
    records.append(record)
            
df = pd.DataFrame(records, columns = head)

In [22]:
df.sample(5)

,team1_name,team2_name,best_of,match_type,date,timestamp,prize,teams,location,map1,map2,map3,map4,map5,team1_player1_name,team1_player1_rating,team1_player1_DPR,team1_player1_KAST,team1_player1_impact,team1_player1_ADR,team1_player1_KPR,team1_player2_name,team1_player2_rating,team1_player2_DPR,team1_player2_KAST,team1_player2_impact,team1_player2_ADR,team1_player2_KPR,team1_player3_name,team1_player3_rating,team1_player3_DPR,team1_player3_KAST,team1_player3_impact,team1_player3_ADR,team1_player3_KPR,team1_player4_name,team1_player4_rating,team1_player4_DPR,team1_player4_KAST,team1_player4_impact,team1_player4_ADR,team1_player4_KPR,team1_player5_name,team1_player5_rating,team1_player5_DPR,team1_player5_KAST,team1_player5_impact,team1_player5_ADR,team1_player5_KPR,team2_player1_name,team2_player1_rating,team2_player1_DPR,team2_player1_KAST,team2_player1_impact,team2_player1_ADR,team2_player1_KPR,team2_player2_name,team2_player2_rating,team2_player2_DPR,team2_player2_KAST,team2_player2_impact,team2_player2_ADR,team2_player2_KPR,team2_player3_name,team2_player3_rating,team2_player3_DPR,team2_player3_KAST,team2_player3_impact,team2_player3_ADR,team2_player3_KPR,team2_player4_name,team2_player4_rating,team2_player4_DPR,team2_player4_KAST,team2_player4_impact,team2_player4_ADR,team2_player4_KPR,team2_player5_name,team2_player5_rating,team2_player5_DPR,team2_player5_KAST,team2_player5_impact,team2_player5_ADR,team2_player5_KPR,team1_points,team2_points,team1_wins,team2_wins
4288,BIG,Unicorns of Love,Best of 3,* Semi-final,2020-05-23,1590256800000,"$37,880",6,Europe (Online),Vertigo,Dust2,Nuke,None,None,tabseN,1.05,0.70,0.697,1.25,84.9,0.74,tiziaN,0.97,0.66,0.700,0.98,72.2,0.65,XANTARES,1.17,0.68,0.721,1.34,89.2,0.82,syrsoN,1.09,0.63,0.701,1.21,74.8,0.74,k1to,1.04,0.69,0.695,1.06,75.1,0.69,crisby,0.94,0.71,0.678,0.98,71.7,0.65,kzy,0.94,0.67,0.691,0.95,72.6,0.64,Andyy,1.04,0.65,0.683,1.01,69.9,0.70,JDC,1.05,0.70,0.688,1.08,77.3,0.70,P4TriCK,0.98,0.70,0.670,0.95,72.4,0.64,2,0,True,False
46,Big Chillin,Recon 5,Best of 3 (Online),* Group B,2021-06-02,1622595600000,"$50,000",18,North America (Online),Inferno,Vertigo,Overpass,None,None,bron,0.98,0.73,0.679,0.95,74.9,0.65,dvl,0.84,0.71,0.645,0.78,62.5,0.53,Munk,1.06,0.70,0.683,1.12,74.8,0.74,Xzen,1.06,0.71,0.703,1.05,79.1,0.71,Elk,1.05,0.74,0.681,1.13,78.9,0.70,desi,1.02,0.70,0.693,1.07,76.0,0.72,AAustiN,0.95,0.69,0.682,0.86,69.2,0.62,Wilky,1.01,0.74,0.684,1.02,76.2,0.70,SATURN,0.98,0.71,0.670,0.99,73.1,0.65,SLIGHT,1.00,0.70,0.684,0.99,73.6,0.67,1,2,False,True
185,PGE Turow,Illuminar,Best of 3 (Online),* Lower bracket semi-final. Winner qualifies f...,2021-05-26,1622061000000,4 Spots at LAN Finals,6+,Poland (Online),Mirage,Dust2,Vertigo,None,None,repo,0.93,0.70,0.675,0.93,69.3,0.65,splawik,0.88,0.73,0.647,0.85,65.6,0.60,blacktear5,0.71,0.77,0.594,0.51,60.3,0.42,suonko,0.78,0.75,0.617,0.70,59.0,0.55,Frontsider,0.78,0.77,0.615,0.83,56.4,0.51,mouz,1.03,0.65,0.719,1.02,74.5,0.69,morelz,0.99,0.70,0.682,1.06,76.8,0.69,GruBy,1.01,0.67,0.708,1.05,78.3,0.69,Sidney,1.00,0.65,0.699,0.92,70.7,0.62,mhL,1.17,0.64,0.708,1.23,80.4,0.77,0,2,False,True
3523,SKADE,forZe,Best of 1,* Group B winners' match,2020-07-01,1593623400000,"$40,000",16,Europe (Online),Nuke,Dust2,Inferno,None,None,SPELLAN,1.03,0.66,0.689,1.02,74.4,0.66,Rainwaker,1.11,0.65,0.725,1.07,78.4,0.71,dennyslaw,1.12,0.65,0.708,1.15,76.0,0.74,oxygeN,1.00,0.65,0.704,0.95,68.4,0.63,Duplicate,1.02,0.68,0.705,0.96,73.4,0.64,facecrack,0.99,0.71,0.693,1.11,77.9,0.70,Jerry,1.07,0.67,0.711,1.04,76.2,0.68,xsepower,1.17,0.58,0.714,1.22,73.0,0.73,almazer,1.06,0.65,0.713,1.00,74.3,0.67,FL1T,1.13,0.65,0.722,1.07,80.1,0.73,0,2,False,True
3671,Apeks,100PG,Best of 1,* Round of 16,2020-06-26,1593203400000,4 spots at Closed Qualifier,16+,Europe (Online),Dust2,None,None,None,None,Marcelious,1.08,0.67,0.692,1.11,74.5,0.72,akez,0.92,0.71,0.683,0.94,71.8,0.63,Grusarn,1.00,0.71,0.681,1.04,73.1,0.66,truth,1.00,0.69,0.693,1.02,75.8,0.70,dennis,1.03,0.68,0.694,1.06,76.6,0.71

In [23]:
df.to_csv("matches.csv")